In [2]:
import tensorflow as tf
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk
import matplotlib.pyplot as plt
import tensorflow as tf

2025-03-01 14:27:20.505327: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-01 14:27:20.524671: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740819440.547701   29550 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740819440.555114   29550 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-01 14:27:20.583897: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = TFT5ForConditionalGeneration.from_pretrained(model_name)


I0000 00:00:1740819516.756161   29550 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 48 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
2025-03-01 14:28:36.781411: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1193] failed to allocate 48.75MiB (51118080 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
2025-03-01 14:28:36.781570: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1193] failed to allocate 43.88MiB (46006272 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
2025-03-01 14:28:36.781651: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1193] failed to allocate 39.49MiB (41405696 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
2025-03-01 14:28:36.781726: I external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:1193] failed to all

ResourceExhaustedError: {{function_node __wrapped__StatelessTruncatedNormalV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[32128,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessTruncatedNormalV2] name: 

In [ ]:
def shift_right(input_ids, pad_token_id, decoder_start_token_id):
    """
    Shifts input ids to the right by one position.
    The first token becomes the decoder_start_token_id.
    """
    batch_size = tf.shape(input_ids)[0]
    start_tokens = tf.fill([batch_size, 1], decoder_start_token_id)
    shifted_input_ids = tf.concat([start_tokens, input_ids[:, :-1]], axis=1)
    return shifted_input_ids

class MultiTaskT5(tf.keras.Model):
    def __init__(self, model_name, num_emotion_labels, num_sentiment_labels):
        super(MultiTaskT5, self).__init__()
        self.t5 = TFT5ForConditionalGeneration.from_pretrained(model_name)
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.emotion_classifier = tf.keras.layers.Dense(
            num_emotion_labels, activation='softmax', name="emotion_classifier"
        )
        self.sentiment_classifier = tf.keras.layers.Dense(
            num_sentiment_labels, activation='softmax', name="sentiment_classifier"
        )

    def call(self, inputs, task, training=False, labels=None):
        if task in ['emotion', 'sentiment']:
            encoder_outputs = self.t5.encoder(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                training=training
            )
            pooled_output = tf.reduce_mean(encoder_outputs.last_hidden_state, axis=1)
            pooled_output = self.dropout(pooled_output, training=training)
            if task == 'emotion':
                return self.emotion_classifier(pooled_output)
            else:
                return self.sentiment_classifier(pooled_output)
        elif task == 'summary':
            if labels is None:
                outputs = self.t5(
                    input_ids=inputs['input_ids'],
                    attention_mask=inputs['attention_mask'],
                    training=training
                )
                return outputs.logits
            else:
                pad_token_id = self.t5.config.pad_token_id
                decoder_start_token_id = self.t5.config.decoder_start_token_id
                decoder_input_ids = shift_right(labels, pad_token_id, decoder_start_token_id)
                outputs = self.t5(
                    input_ids=inputs['input_ids'],
                    attention_mask=inputs['attention_mask'],
                    decoder_input_ids=decoder_input_ids,
                    training=training
                )
                return outputs.logits
        else:
            raise ValueError("Unsupported task type. Use 'emotion', 'sentiment', or 'summary'.")

In [ ]:
model_name = "t5-small"
num_emotion_labels = 6
num_sentiment_labels = 4  # Updated to 4 output labels for sentiment
model = MultiTaskT5(model_name, num_emotion_labels, num_sentiment_labels)
tokenizer = T5Tokenizer.from_pretrained(model_name)
        


In [ ]:
# Load weights saved in H5 format.
model.load_weights('./results/multi_task_T5_tf_weights')


In [ ]:
# Load the tokenizer from the saved directory.
tokenizer = T5Tokenizer.from_pretrained("./results/multi_task_T5_tf/")


In [ ]:
sentiment_mapping = {3: 'positive', 2: 'neutral', 1: 'negative', 0: 'irrelevant'}

emotion_mapping = {
    0: "anger", 
    1: "fear", 
    2: "joy", 
    3: "love", 
    4: "sadness", 
    5: "surprise"
}

def decode_sentiment(label_code):
    return sentiment_mapping.get(label_code, "Unknown")

def decode_emotion(label_code):
    return emotion_mapping.get(label_code, "Unknown")

In [ ]:
def predict(text, task):
    if task in ['emotion', 'sentiment']:
        # Tokenize the input with a moderate max_length.
        inputs = tokenizer(text, return_tensors="tf", max_length=128, truncation=True, padding="max_length")
        logits = model(inputs, task=task, training=False)
        # Get the predicted label index.
        pred_idx = int(tf.argmax(logits, axis=1).numpy()[0])
        # Decode the prediction using our mapping functions.
        if task == "sentiment":
            return decode_sentiment(pred_idx)
        elif task == "emotion":
            return decode_emotion(pred_idx)
    
    elif task == 'summary':
        # Prepend the summarization prefix as required by T5.
        input_text = "summarize: " + text
        # Tokenize with a longer max_length for summarization inputs.
        inputs = tokenizer(input_text, return_tensors="tf", max_length=512, truncation=True, padding="max_length")
        # Use the underlying T5 model's generate function.
        generated_ids = model.t5.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=150,
            num_beams=4,
            early_stopping=True
        )
        # Decode the generated token ids to text.
        summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        return summary
    
    else:
        raise ValueError("Unsupported task type. Use 'emotion', 'sentiment', or 'summary'.")


In [ ]:
# Test sentiment classification
sentiment_result = predict("smart guy, i would surely hire", task="sentiment")
print("Sentiment Prediction:", sentiment_result)

# Test emotion classification
emotion_result = predict("smart guy, i would surely hire", task="emotion")
print("Emotion Prediction:", emotion_result)

# Test summarization
summary_result = predict("This is an example text that we want to summarize.", task="summary")
print("Summary Output:", summary_result)


In [ ]:
smart guy, i would surely hire